In [ ]:
%load_ext dotenv
%dotenv .env

In [ ]:
from uuid import uuid4

from langchain_core.runnables.graph import MermaidDrawMethod
from langfuse.callback import CallbackHandler

from redbox.app import Redbox
from redbox.models.chain import AISettings, ChatLLMBackend, RedboxQuery, RedboxState
from redbox.models.settings import get_settings

In [ ]:
def get_state(user_uuid, prompts, documents, ai_setting):
    q = RedboxQuery(
        question=f"@search {prompts[-1]}",
        s3_keys=documents,
        user_uuid=user_uuid,
        chat_history=prompts[:-1],
        ai_settings=ai_setting,
        permitted_s3_keys=documents,
    )

    return RedboxState(
        request=q,
    )


def run_app(app, state) -> RedboxState:
    langfuse_handler = CallbackHandler()
    return app.graph.invoke(state, config={"callbacks": [langfuse_handler]})

In [ ]:
# Basic configuration
env = get_settings()
ai_setting = AISettings(chat_backend=ChatLLMBackend(name="anthropic.claude-3-sonnet-20240229-v1:0", provider="bedrock"))
app = Redbox(debug=True, env=env)
documents = ["adela.iliescu@digital.trade.gov.uk/179._Design_Council.pdf"]
user_prompt = ["How does the exposure to AI vary across different geographical areas?"]

x = get_state(uuid4(), prompts=[user_prompt], documents=documents, ai_setting=ai_setting)
result = run_app(app, x)

In [ ]:
# Single question
env = get_settings()
ai_setting = AISettings(chat_backend=ChatLLMBackend(name="anthropic.claude-3-sonnet-20240229-v1:0", provider="bedrock"))
app = Redbox(debug=True, env=env)
documents = ["adela.iliescu@digital.trade.gov.uk/1 The power chapter.pdf"]
user_prompt = "How does the exposure to AI vary across different geographical areas?"

llm_rephrased = []
final_answer = []
for _i in range(5):
    x = get_state(uuid4(), prompts=[user_prompt], documents=documents, ai_setting=ai_setting)
    result = run_app(app, x)
    llm_rephrased.append(result["messages"][0].content)
    final_answer.append(result["messages"][1].content)

In [ ]:
# print(f"""User prompt: {user_prompt};\n
#       Rephrase 1: {llm_rephrased[0]};\n
#       Rephrase 2: {llm_rephrased[1]};\n
#       Rephrase 3: {llm_rephrased[2]};\n
#       Rephrase 4: {llm_rephrased[3]};\n
#       Rephrase 5: {llm_rephrased[4]}""")

In [ ]:
# print(f"""Answer 1: {final_answer[0]};\n
#       Answer 2: {final_answer[1]};\n
#       Answer 3: {final_answer[2]};\n
#       Answer 4: {final_answer[3]};\n
#       Answer 5: {final_answer[4]}""")

In [ ]:
# Several questions test (might take a while)
env = get_settings()
ai_setting = AISettings(chat_backend=ChatLLMBackend(name="anthropic.claude-3-sonnet-20240229-v1:0", provider="bedrock"))
app = Redbox(debug=True, env=env)
documents = ["adela.iliescu@digital.trade.gov.uk/179._Design_Council.pdf"]
user_prompt_list = [
    "How does the exposure to AI vary across different geographical areas?",
    "According to the US Bureau of Labor Statistics data released in 2018, "
    "what percentage of those in “computer and mathematical occupations” "
    "were women",
    "What are the four domains of power within Patricia Hill Collins's matrix of domination",
    "What are the main points?",
    "What are some issues in data science?",
    "Why is data important",
]
# Separate dictionaries for rephrased prompts and answers
results_prompt = {}
results_answer = {}

for user_prompt in user_prompt_list:
    # Initialize empty lists for each user prompt
    results_prompt[user_prompt] = []
    results_answer[user_prompt] = []

    for _i in range(5):
        x = get_state(uuid4(), prompts=[user_prompt], documents=documents, ai_setting=ai_setting)
        result = run_app(app, x)

        results_prompt[user_prompt].append(result["messages"][0].content)
        results_answer[user_prompt].append(result["messages"][1].content)

In [ ]:
results_prompt

In [ ]:
%%capture
# Several questions test (might take a while)
env = get_settings()
ai_setting = AISettings(chat_backend=ChatLLMBackend(name="anthropic.claude-3-sonnet-20240229-v1:0", provider="bedrock"))
app = Redbox(debug=True, env=env)
documents = ["adela.iliescu@digital.trade.gov.uk/179._Design_Council.pdf"]
user_prompt_list = [
    "What industries are most exposed to Large Language Models (LLM) according to the report?",
    "How does the report define and assess AI application-ability relatedness "
    "in its methodology for calculating occupational exposure?",
    "What training routes are associated with the highest exposure to AI for early-career employees?",
    "What is the impact of AI on jobs?",
    "Tell me about AI in the UK",
]
# Separate dictionaries for rephrased prompts and answers
results_prompt = {}
results_answer = {}

for user_prompt in user_prompt_list:
    # Initialize empty lists for each user prompt
    results_prompt[user_prompt] = []
    results_answer[user_prompt] = []

    for _i in range(5):
        x = get_state(uuid4(), prompts=[user_prompt], documents=documents, ai_setting=ai_setting)
        result = run_app(app, x)

        results_prompt[user_prompt].append(result["messages"][0].content)
        results_answer[user_prompt].append(result["messages"][1].content)

In [ ]:
results_prompt

In [ ]:
graph = app.graph.get_graph()
graph.draw_mermaid_png(draw_method=MermaidDrawMethod.API, output_file_path="./graph.png")

In [ ]:
graph = app.graph.get_graph().nodes["p_retrieve_metadata"].data.get_graph()
graph.draw_mermaid_png(draw_method=MermaidDrawMethod.API, output_file_path="./graph.png")

In [ ]:
graph = app.graph.get_graph().nodes["p_search"].data.get_graph()
graph.draw_mermaid_png(draw_method=MermaidDrawMethod.API, output_file_path="./graph.png")